In [1]:
from IPython.core.display import display, HTML
display(HTML("""<style> .container {width:96% !important;}</style>"""))

from IPython.display import IFrame

In [2]:
import pandas as pd
import numpy as np
# from plotly.offline import init_notebook_mode, iplot
# import cufflinks as cf
# init_notebook_mode()
# cf.go_offline()
from __future__ import division

In [3]:
import sys
sys.path.insert(0,'../')
from utils.paths import *

![](asset/role of SBA.png)

# Business case

SBA acts much like an insurance provider to reduce the risk for a bank by taking on some of the risk through guaranteeing a portion of the loan. 

* What is the risk level of SBA?
* What is the overall loss ratio of SBA?
* Can we improve it using machine learning?

## Experiment: 

1. Study the loss ratio in 2015
2. Use 2003 and 2004 data to build a machine learning model
3. Use this model to predict default in 2005


In [69]:
nat = pd.read_csv(path_SBA + 'SBAnational_new.csv', sep = ';', low_memory=False)

In [70]:
nat5 = nat[nat.ApprovalFY.isin([2005])].reset_index(drop = True)

## Assign grade based on SBA ratio

Let's assign grades for SBA based on the SBA ratio, 
i.e. grade 5 for below 0.2, grade 4 for ratio between 0.2 to 0.4

In [71]:
sba_ratio_th = [0, 0.2, 0.4, 0.6, 0.8, 1]
sba_grades = [5, 4, 3, 2, 1]

nat5['SBA_grades'] = pd.cut(nat5.SBA_ratio, bins = sba_ratio_th, labels = sba_grades)

In [72]:
nat5.SBA_grades.value_counts().sort_index(ascending = False)

1    17294
2    10931
3    48706
4       23
5        3
Name: SBA_grades, dtype: int64

In [73]:
def default_cat_table(data, cat):
    default_cat = data.groupby([cat, 'default']).count().max(1).unstack()
    default_cat['ALL'] = data[cat].value_counts()
    default_cat['default_rate'] = (default_cat[1] / default_cat['ALL'])
    default_cat = default_cat.rename(columns = {1: 'Default', 0: 'Non-default'})
    return default_cat


In [74]:
default_cat_table(nat5, 'SBA_grades').sort_index(ascending = False)

default,Non-default,Default,ALL,default_rate
SBA_grades,,,,
1,13285.0,4009.0,17294,0.231815
2,8999.0,1932.0,10931,0.176745
3,35169.0,13537.0,48706,0.277933
4,22.0,1.0,23,0.043478
5,3.0,NaN,3,NaN


In [76]:
def sba_claim(chgoffamount, sba_app):
    if chgoffamount == 0:
        return 0
    else:
        if sba_app <= chgoffamount:
            claim = sba_app
        else:
            claim = chgoffamount
        return claim
    
claim = nat5.apply(lambda x: sba_claim(x['ChgOffPrinGr'], x['SBA_Appv']), axis = 1)
nat5['SBA_claim'] = claim

In [87]:
print 'Overall lost ratio = ', round(nat5.SBA_claim.sum() / nat5.SBA_Appv.sum(), 3)

print 'Lost ratio for each grade: '

SBA_grade_df = pd.DataFrame(nat5.groupby('SBA_grades').GrAppv.sum())
SBA_grade_df['SBA_Appv'] = nat5.groupby('SBA_grades').SBA_Appv.sum()
SBA_grade_df['SBA_claim'] = nat5.groupby('SBA_grades').SBA_claim.sum()
SBA_grade_df['SBA_loss_ratio'] = SBA_grade_df.SBA_claim / SBA_grade_df.SBA_Appv
SBA_grade_df.sort_index(ascending = False)

Overall lost ratio =  0.113
Lost ratio for each grade: 


,GrAppv,SBA_Appv,SBA_claim,SBA_loss_ratio
SBA_grades,,,,
1,3.182030e+09,3.051684e+09,120698864.0,0.039552
2,5.645468e+09,4.221814e+09,533314200.0,0.126323
3,2.759550e+09,1.383032e+09,324853975.0,0.234885
4,1.586210e+07,4.916853e+06,284994.0,0.057963
5,1.907000e+06,2.552700e+05,0.0,0.000000
